In [ ]:
!pip install -U sentence-transformers

In [3]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

In [9]:
!python.exe -m pip install --upgrade pip

  Using cached pip-25.0.1-py3-none-any.whl.metadata (3.7 kB)
Using cached pip-25.0.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 25.0
    Uninstalling pip-25.0:
      Successfully uninstalled pip-25.0


In [ ]:
from sentence_transformers import SentenceTransformer, util

In [4]:
# 필요한 라이브러리 설치 (최초 1회 실행)
!pip install -q sentence-transformers openpyxl

from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 1. 모델 로드
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 2. 데이터 로드
# symptom_data: 증상 관련 데이터, 'text' (증상 텍스트)와 '건강 카테고리' 컬럼 포함
# material_data: 제품 관련 데이터, '품목명', '건강 카테고리', '주요 기능', '섭취 주의사항' 컬럼 포함
symptom_data = pd.read_excel("C:\\Users\\dbwld\\OneDrive\\바탕 화면\\Project\\symptom_data.xlsx")
material_data = pd.read_excel("C:\\Users\\dbwld\\OneDrive\\바탕 화면\\Project\\combined_data.xlsx")

# 3. 사용자 입력 및 임베딩 계산
user_symptom = "스트레스를 많이 받아요."
user_embedding = model.encode(user_symptom)

# 4. 증상 데이터에서 사용자 증상과 유사한 증상 찾기
symptom_similarities = []
for idx, row in symptom_data.iterrows():
    symptom_text = row['text']
    # 각 증상의 임베딩 계산
    symptom_embedding = model.encode(symptom_text)
    # 코사인 유사도 계산 (입력은 2차원 배열)
    sim = cosine_similarity([user_embedding], [symptom_embedding])[0][0]
    symptom_similarities.append((sim, symptom_text, row['건강 카테고리']))

# 유사도가 높은 상위 2개 증상 선택
top_similar_symptoms = sorted(symptom_similarities, key=lambda x: x[0], reverse=True)[:2]
print("Top similar symptoms:", top_similar_symptoms)

# 5. 상위 증상에서 추출한 건강 카테고리 사용
similar_categories = [item[2] for item in top_similar_symptoms]

# 6. 제품 데이터에서 해당 건강 카테고리에 속하는 제품 중 유사도 높은 제품 추천
recommended_products = []
for idx, row in material_data.iterrows():
    if row['건강 카테고리'] in similar_categories:
        product_embedding = model.encode(row['주요 기능'])
        sim_score = cosine_similarity([user_embedding], [product_embedding])[0][0]
        recommended_products.append((row['품목명'], sim_score, row['섭취 주의사항']))

# 유사도가 높은 상위 3개 제품 선택
recommended_products = sorted(recommended_products, key=lambda x: x[1], reverse=True)[:3]
print("추천 제품 (유사도 기준):", recommended_products)


#지병 및 특이사항 제외하는 코드
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
# 6. 사용자가 입력한 지병 또는 특이사항에 대해 제품의 섭취 주의사항이 안전한지 판단하는 함수
# 6. 각 번호별로 안전 여부를 개별 평가하는 함수
def is_safe_for_condition(product_warning, user_condition, threshold=0.45):
    """
    product_warning: 제품의 섭취 주의사항 전체 텍스트 (여러 항목이 '\n'으로 구분되어 있다고 가정)
    user_condition: 사용자가 입력한 지병 또는 특이사항 (예: "알레르기가 있음")
    threshold: 각 항목과 사용자의 입력 간 코사인 유사도의 임계값
    
    반환: 모든 항목에 대해 유사도가 threshold 미만이면 True, 
          하나라도 threshold 이상이면 False로 판단 (해당 조건과 관련이 있다고 봄)
    """
    # 줄바꿈을 기준으로 각 항목 분리
    bullets = product_warning.split('\n')
    if len(bullets) == 0:
        bullets = [product_warning]
    for bullet in bullets:
        bullet = bullet.strip()
        if bullet == "":
            continue
        sim = util.cos_sim(model.encode([bullet]), model.encode([user_condition]))[0][0].item()
        # 하나라도 사용자의 조건과 유사도가 임계값 이상이면, 해당 제품은 안전하지 않다고 판단
        if sim >= threshold:
            return False
    return True

# 7. 사용자가 입력한 조건 (예: "알레르기가 있음")
input_disease = "임산부에요"

# 8. 추천 제품 중, 각 항목별로 사용자 입력값과 비교하여 안전한 제품만 최종 추천 리스트에 포함
final_recommendations = []
for prod_name, score, warning in recommended_products:
    if is_safe_for_condition(warning, input_disease, threshold=0.45):
        final_recommendations.append(prod_name)

print("최종 추천 제품:", final_recommendations)

Top similar symptoms: [(0.34735554, '정신이 멍하고 맑지 않아서 집중하기가 힘들어요. 자주 일을 잊어버리고 기억하는 데 어려움을 겪고 있어요. 집중하려고 해도 자꾸 산만해지고 주의를 오래 유지하기가 어려워요.', '정신 및 건강'), (0.32731092, '요즘 몸이 너무 피곤하고, 하루 종일 쉬어도 전혀 회복된 느낌이 없어요. 조금만 움직여도 너무 지쳐서 몸이 힘들어요. 기억력이 떨어지고, 생각도 잘 안 나고 집중이 안 돼요. 누워 있다가 일어나면 어지러움이 심해지고, 근육이나 관절도 자주 아파요. 잠을 자도 전혀 개운하지 않고, 피로가 계속 쌓이는 것 같아요.\n가끔은 머리가 아프고, 목이 아프거나 부어오르기도 해요. 목이나 겨드랑이 림프절도 아프고 예민해요. 빛이나 소리, 냄새, 음식, 약물에도 예민해져서 더 불편해져요.', '수면 및 정신')]
추천 제품 (유사도 기준): [('유단백가수분해물', 0.49488497, '(가) 어린이나 임산부, 수유부는 섭취를 주의해야 함\n(나) 충분한 물과 함께 섭취해야 함\n(다) 과다 섭취 시 미약한 설사, 구토, 배변량 증가, 배변빈도 증가, 복부 팽만, 두통 등의 부작용을 일으킬 수 있음'), ('돌외잎추출물(기능성원료인정제2015-7호)', 0.46292204, '(1) 과량 섭취 시 위장장애(구토와 위장운동 증가)가 발생할 수 있음 (2) 아스피린, 혈액항응고제 복용 시 섭취에 주의'), ('돌외잎추출물(제2015-7호)', 0.46292204, '①기관지 천식, 해소 기침이나 기도에 심한 염증이 있는 사람은 사용을 금하시기 바랍니다.')]
최종 추천 제품: ['유단백가수분해물', '돌외잎추출물(기능성원료인정제2015-7호)', '돌외잎추출물(제2015-7호)']
